In [97]:
import numpy as np;

In [98]:
with open("movie-xids.npy","rb") as f:
    Xids=np.load(f,allow_pickle=True)
with open("movie-xmask.npy","rb") as f:
    mask=np.load(f,allow_pickle=True)
with open("movie_labels.npy","rb") as f:
    labels=np.load(f,allow_pickle=True)

In [99]:
import tensorflow as tf;
dataset=tf.data.Dataset.from_tensor_slices((Xids,mask,labels)).shuffle(10000).batch(16,drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [100]:
dataset.take(1)

<TakeDataset shapes: ((16, 512), (16, 512), (16, 5)), types: (tf.int32, tf.int32, tf.float64)>

In [101]:
def map_func(Xids,mask,labels):
    return {'input_ids':Xids,'attention_mask':mask},labels

In [102]:
dataset=dataset.map(map_func)

In [103]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [104]:
split=0.8
size=int((Xids.shape[0]/16)*split)
train_ds=dataset.take(size)
val_ds=dataset.skip(size)

In [105]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [106]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [107]:
train_ds.element_spec==val_ds.element_spec

True

In [108]:
tf.data.experimental.save(train_ds,'train')
tf.data.experimental.save(val_ds,'val')